***Dependencies***: Only necessary for initially running notebook code.

In [ ]:
%pip install azure-ai-ml
%pip install -U transformers==4.20
%pip install -U tensorflow==2.9


***Connect to workspace***: Run this block of code when beginning session. This will connect the notebook to the workspace which contains all resources and artifacts needed for deploying the model. 

In [ ]:
# imports
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

# connect to the workspace
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AzureML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AZUREML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)

***Create environment***: This will create the containerised environment for deployment. It will take a docker image along with conda file, containing all necassary dependencies. To update dependencies, edit the conda.yml file and rerun code block below.

In [ ]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/minimal-ubuntu20.04-py38-cpu-inference:latest",
    conda_file="src/conda.yml",
    name="jam_environment",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

***Create model ***: This will train the model and output two model files in src/deployment/model. The hf_model.h5 is the hugging face base model file and custom_bert.h5 is the fine-tuned model file. This will not run the experiment as an Azure job but is sufficient for creating deployment. 

In [ ]:
# Imports
import pandas as pd 
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf

# Parameters
BATCH_SIZE = 16
EPOCHS = 1
CLEAN_TEXT = False
ADD_DENSE = False
DENSE_DIM = 64
ADD_DROPOUT = False
DROPOUT = .2
TRAIN_BASE = True

# Data specific fields
textCol = "text"
clusterCol = "label"


def bert_encode(data, maximum_len):
    input_ids = []
    attention_masks = []

    for iterator in range(len(data.text)):
        encoded = tokenizer.encode_plus(data.text.iloc[iterator],
                                        add_special_tokens = True,
                                        max_length = maximum_len,
                                        pad_to_max_length = True,
                                        return_attention_mask = True)

        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    
    return np.array(input_ids), np.array(attention_masks)


def build_model(model_layer, learning_rate, add_dense = ADD_DENSE,
                dense_dim = DENSE_DIM, add_dropout = ADD_DROPOUT, dropout = DROPOUT):
    
    # define inputs
    input_ids = tf.keras.Input(shape = (128,), dtype ='int32')
    attention_masks = tf.keras.Input(shape =(128,), dtype ='int32')

    # insert BERT layer
    sequence_output = model_layer(input_ids, attention_masks)

    # choose only last hidden state
    output = sequence_output[0]
    output = output[:,0,:]

    output = tf.keras.layers.Dense(32, activation = 'relu')(output)
    output = tf.keras.layers.Dense(7, activation = 'softmax')(output) # changed the last layer to 7 from 5 since we have 7 clusters in our data

    # assemble and compile

    model = tf.keras.models.Model(inputs = [input_ids, attention_masks], outputs = output)
    model.compile(tf.keras.optimizers.Adam(learning_rate = learning_rate),
                  loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model


# import data and rename fields - HSBC needs to change this bit to work with their data
train = pd.read_csv('src/data/train.csv')
train = train[[textCol, clusterCol]]
train.rename(columns={textCol: 'text', clusterCol: 'label'}, inplace=True)

# Import models (from folder in your Azure directory)
pathToHFModels = 'src/deployment/hf_model' # update this path to where you are storing the model
tokenizer = AutoTokenizer.from_pretrained(pathToHFModels)
model = TFAutoModel.from_pretrained(pathToHFModels)

# Tokenize
if TRAIN_BASE:
    train_inputs_ids, train_attention_masks = bert_encode(train, 128)

# Build the model
BERT_base = build_model(model, learning_rate = 1e-5)

# Create a checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('src/deployment/model/hf_model.h5',
    monitor = 'val_loss', save_best_only = True, save_weights_only = True)

# Train the model
if TRAIN_BASE:
    history = BERT_base.fit([train_inputs_ids, train_attention_masks], train.label,
                            validation_split = .2, epochs = EPOCHS, callbacks = [checkpoint])

# Save the trained model locally
BERT_base.save('src/deployment/model/custom_bert.h5') # update this path to where you wish to save the model

***Run model training as an Azure experiment/job***

In [ ]:
from azure.ai.ml import command
from azure.ai.ml import Input
import mlflow

job = command(
    code="./src/",  # location of source code
    command="python jam_model.py",
    environment="jam_environment:1",
    compute="dg-compute1",
    experiment_name="train_model",
    display_name="jam-job",
)

# submit the command
ml_client.jobs.create_or_update(job)

***Register the model***

In [ ]:
# register model from inline script

from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

# register the model
model = Model(
    path="src/deployment",
    type=AssetTypes.CUSTOM_MODEL,
    name="deployment_jam",
    description="Model created with dummy data to test the process",
)

ml_client.models.create_or_update(model)

In [ ]:
# register the model from the job

from azure.ai.ml.entities import Model

model = Model(
    # the script stores the model as "model"
    path="azureml://jobs/{}/outputs/artifacts/paths/outputs/model/".format(
        "dynamic_oil_vy2vpwjjds"
    ),
    name="run_model_example",
    description="Model created from run.",
    type="custom_model",
)

registered_model = ml_client.models.create_or_update(model=model)

***Create endpoint for deployment***

In [ ]:
# create endpoint

import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "endpoint-" + str(uuid.uuid4())[:8]

from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
)

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="",
    auth_mode="key",
)

endpoint = ml_client.begin_create_or_update(endpoint)

endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpoint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved')


***Register model to endpoint***

In [ ]:
# deploy model to endpoint

model = 'deployment_jam:1'

from azure.ai.ml.entities import CodeConfiguration

# create an online deployment.
jam_deployment = ManagedOnlineDeployment(
    name="JAM-deployment",
    endpoint_name=online_endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(code="src", scoring_script="score_realtime.py"),
    environment="jam_environment:3",
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

jam_deployment = ml_client.begin_create_or_update(jam_deployment)

***Test deployment***

In [ ]:
import json
import pandas as pd
from azure.ai.ml import MLClient

# predict using the deployed model
result = ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="src/data/test.txt",
    deployment_name="jam-deployment",
)

results = json.loads(result)
result_data = pd.DataFrame(eval(results))
result_data